# Top End Assembly Vibration Study

As described in [OBS-467], we have been noticing strong vibrations on the Top End Assembly (TEA) of the Simonyi Survey Telescope (SST). 
The first time we saw these vibrations was on March 9th (Saturday) and on March 10th (Sunday). 

Accordingly to [SITCOM-1285], on March 15th (Friday), we ran soak tests using the Rotator at different configurations. 
We want to investigate any possible vibrations during that period to confirm the hypothesis that the Rotator could be causing this vibration.

In addition to both cases, on Marth 21th (Thursday), we executed [BLOCK-197] a couple of times. 
[OBS-498] reports multiple faults on the hexapods associated with the compensation mode. 
It also indicates that we might have vibrations during its execution, we need to review it. 

Finally, on March 25th, we executed [BLOCK-197] twice. 
First with the rotator in ENABLED. 
Second with the rotator in STANDBY and with its cabinet turned off. 
We want to compare both cases. 

[BLOCK-197]: https://rubinobs.atlassian.net/browse/BLOCK-197
[OBS-467]: https://rubinobs.atlassian.net/browse/OBS-467
[OBS-498]: https://rubinobs.atlassian.net/browse/OBS-498
[SITCOM-1285]: https://rubinobs.atlassian.net/browse/SITCOM-1285

## Notebook Preparation

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
import asyncio
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
import time
import warnings

from argparse import Namespace
from astropy.time import Time, TimeDelta
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst_efd_client.efd_helper import merge_packed_time_series

# Ignore the many warning messages from ``merge_packed_time_series``
warnings.simplefilter(action="ignore", category=FutureWarning)

# Create an EFD client
client = makeEfdClient()

# Global variables definition
baseFields = ["accelerationX", "accelerationY", "accelerationZ"]
sensorNames = [
    "SST top end ring +x -y",
    "SST top end ring -x -y",
    "SST spider spindle",
    "SST M2 surrogate",
]

## OBS-467 - The initial problem

In [ ]:
start = Time("2024-03-09T10:00:00Z", scale="utc", format="isot")
end = Time("2024-03-11T06:00:00Z", scale="utc", format="isot")

df_rot = getEfdData(
    client,
    "lsst.sal.MTRotator.rotation",
    columns=["actualPosition"],
    begin=start,
    end=end,
)

df_m2 = getEfdData(
    client,
    "lsst.sal.MTM2.positionIMS",
    columns=["x", "y", "z"],
    begin=start,
    end=end,
)

df_hex = getEfdData(
    client,
    "lsst.sal.MTHexapod.application",
    columns=[f"position{i}" for i in range(3)] + ["salIndex"],
    begin=start,
    end=end,
)

In [ ]:
df_m2 = df_m2 - df_m2.rolling("1s").mean()
df_camhex = df_hex[df_hex.salIndex == 1]
df_m2hex = df_hex[df_hex.salIndex == 2]

<div class="alert alert-warning">
The plot below takes a 2.5 min to be executed. 
</div>

In [ ]:
%matplotlib inline
fig, (ax_m2, ax_camhex, ax_m2hex, ax_rot) = plt.subplots(
    nrows=4, figsize=(16, 7), sharex=True
)

ax_m2.plot(df_m2.x, label="x")
ax_m2.plot(df_m2.y, label="y")
ax_m2.plot(df_m2.z, label="z")
ax_m2.grid(":", alpha=0.25)
ax_m2.set_ylabel("M2 IMS\n Position [um]")
ax_m2.legend(loc="upper left")

ax_camhex.plot(df_camhex.position0, label="x")
ax_camhex.plot(df_camhex.position1, label="y")
ax_camhex.plot(df_camhex.position2, label="z")
ax_camhex.grid(":", alpha=0.25)
ax_camhex.set_ylabel("CamHex\n Position [um]")
ax_camhex.legend(loc="upper left")

ax_m2hex.plot(df_m2hex.position0, label="x")
ax_m2hex.plot(df_m2hex.position1, label="y")
ax_m2hex.plot(df_m2hex.position2, label="z")
ax_m2hex.grid(":", alpha=0.25)
ax_m2hex.set_ylabel("M2\n Position [um]")
ax_m2hex.legend(loc="upper left")

ax_rot.plot(df_rot.actualPosition * 1e3, label="Actual Position")
ax_rot.grid(":", alpha=0.25)
ax_rot.set_ylabel("Rotator Angle\n [$10^-3$ deg]")
ax_rot.set_xlim(df_rot.index[0], df_rot.index[-1])
ax_rot.set_xlabel("Time [utc]")
ax_rot.legend(loc="upper left")

fig.suptitle(
    "OBS-467 Investigation - M2/Rotator/Hexapods Vibrrations\n"
    f"From {start.to_value('iso', subfmt='date_hm')} to {end.to_value('iso', subfmt='date_hm')}"
)

fig.autofmt_xdate()
fig.tight_layout()

fig.savefig("obs467_investigation.png")
plt.show()

The plots above show a low amplitude vibration on the Rotator and on M2 from 00:40 to 15:00 UTC on March 10th.  
Following the logs in OLE, the first problematic measurements using the Laser Tracker started to appear around 00:44. 
This is consistent with what we see above. 

We checked the vibrations on the rotator actual position on Chronograf, we found that the exact time that it started was at `00:37:55`.
At `14:41:13`, the amplitude of these oscillations are smaller and last until `16:08:41`.  

Alysha confirmed that all the vibrations/oscillations after 17h UTC come from other tests and are unrelated to the sounds we hear when going to Level 8. 

Another important aspect to consider are the amplitudes of the vibrations. 
A quick eye inspection show that the oscillation on the rotator have an amplitude of about 0.1 x 10^-3 degrees. 
The M2 have oscillations around 1 um. 
None of these two justify the 2 mm displacement that we were measuring with the Laser Tracker.

When looking at the hexapods data, we can see the warm-up sequence twice. 
First, at around 18h UTC on March 9th and, second, near 14h UTC on March 10th. 
These two data sets give us an example of the range of movement on the hexapods. 

We can also see both hexapods being exercised quite a lot near the time when the vibrations/oscillations started. 

Several questions remain for this time window and we can use the following events/commands to investigate a bit more:
- summary states for the hexapods and the rotator
- drive states for the hexapods and the rotator
- when was the compensation mode enabled/disabled?